# Imports

In [1]:
# Own Packages
from Masterarbeit_utils.model_utils import get_tokenizer, load_and_modify_model, load_pretrained_Tokenizer

# Site-Packages
import dask.dataframe as dd
import torch
import psutil
import os
import sys
import pickle as pk
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, OPTForCausalLM
from tokenizers.processors import TemplateProcessing
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from torch.utils.data import Dataset

/home/worker/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.executable

'/usr/bin/python3'

# Parameters

In [ ]:
"""
The Paths to important folders have to be changed for your system.
"""

# This folder will be created and filled with txt.files for each sample after you run the Pytorch Dataset Notebook
dataset_folder = f'data/dataset_samples'

# The folder at which the model will be saved. This folder has to be created for your system 
model_folder = f'data/models/gal_125_1'
os.makedirs(model_folder, exist_ok=True)

# The folder at which the training progress will be logged
log_folder = 'data/models/gal_125_1/logs'
os.makedirs(log_folder, exist_ok=True)

# Folder at which all pickle files are stored. This folder is fixed for this project and should not be changed
dump_dir = r'PK_DUMP'

# Model parameters 
'''
mini	125 M
base	1.3 B
standard	6.7 B
large	30 B
huge	120 B'''
base_model_name = 'mini'

# All new Torch-objects will be by default in this dtype
default_dtype = torch.float16
torch.set_default_dtype(default_dtype)

# Default device on which the model will be loaded
default_device = 'cuda:0'

# Number of GPUs the model will be parallelised to 
num_gpus = 1
# If you change 'default_device' to 'cpu', make sure to set num_gpus to zero.
if default_device == 'cpu':
    num_gpus = 0

tensor_parallel = False
n_f_terms = None # Will be calculated

# Training parameters!
output_dir=model_folder
num_train_epochs=3
per_device_train_batch_size=10
per_device_eval_batch_size=10
save_strategy="epoch"
logging_strategy="steps"
evaluation_strategy="steps"
eval_steps = 1000000 # eval_steps = number of samples until evaluation is performed.
eval_steps = int(eval_steps/per_device_train_batch_size)
learning_rate=5e-5
weight_decay=0.0
seed = 42

# This that could improve performance
dataloader_num_workers = 4
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
torch_compile = True
# V-Ram reduction only if default_dtype= float32
fp16=False

# Creating the Tokenizer

In [ ]:
# Loads a pretrained Tokenizer for the galactica model and adds an additional token for each F-Term
tokenizer = get_tokenizer(dump_dir)

# The Tokenizer contained initially 50000 Tokens which are stored as the vocab-size.
# The vocab_size attribute is not updated when the additional tokens are added to the tokenizer
n_f_terms = len(tokenizer) - tokenizer.vocab_size
print(f'There are {n_f_terms} different F-Terms in the whole Dataset!')

# Creating the dataset

In [ ]:
# Samples in train 6385601
# Samples in val 1596401

class JapPatDataset(Dataset):
    """Dataset containing Japanese patents and their F-Term classification"""
    def __init__(self, data_folder, tokenizer):
        """
        data_folder: path to folder containing the text samples
        tokenizer: tokenizer instance with added additional Tokens for F-Terms
        """
        super(Dataset).__init__()
        self.data_folder = data_folder
        # This has to be manually set to the ammount of files in the 'dataset_samples' folder. Calculating the number of files in this folder would take forever.
        # A to low number would lead to samples missing from the dataset.
        # A to high number would raise a FileNotFound error.
        self.l = len(os.listdir(data_folder))
        self.tokenizer = tokenizer
        
    def __len__(self):
        return self.l
    
    def __getitem__(self, idx):
        try:
            with open(f'{self.data_folder}/{idx}.txt', 'r', encoding='utf-8') as f:
                item = f.read()
        except FileNotFoundError:
            raise FileNotFoundError
        
        # Tokenizing the item 
        # The Tokenizer will return a dict with the encoded text as 'input_ids', 
        # a mask which shows the tokens types this will not be needed for our applications
        # and a mask for the attention mechanism as 'attention_mask' The attention mask will be needed to indicate, that the 
        # model should not attend to <pad> tokens.
        return self.tokenizer(item)  

In [ ]:
train_dataset = JapPatDataset(f'{dataset_folder}/train', tokenizer)
validation_dataset = JapPatDataset(f'{dataset_folder}/validation', tokenizer)

In [ ]:
# The pretrained model is loaded from Huggingface.
# The token-embedding is expanded for all f-terms and the output embeddings is compleatly replaced by a F-Term classification head.
model = load_and_modify_model(base_model_name, default_dtype, tensor_parallel, num_gpus, n_f_terms, default_device)
print(f'The model interprets token-index {model.config.bos_token_id} as the beginning of a sequence and {model.config.eos_token_id} as the end')

In [ ]:
# Input Text
text = 'Good morning Mr.'
# Convert text to tokens
tokens  = tokenizer(text, return_tensors='pt').input_ids
print(f'Output of Tokenizer: {tokens}')
# Model generating the predicted output tokens
out = model.generate(tokens.to(default_device), max_length=30)
# Decoding the tokens

out = tokenizer.decode(out[0])
out

# Creating the Trainer Class by Subclassing from Huggingface-Trainer

In [ ]:
# Subclassing the Huggingface Trainer class to use custome code to calculate the loss
# The labels used for the loss are generated and the labels for the text tokens are set to -100 to ignore their loss,
# because the modified model can't predict text-tokens
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs: bool=False):
        """
        model: model which should be trained.
        inputs: A padded batch of samples from the dataset.
        return_outputs: Indicates if the whole output of the model is returned or not.
        """
        # Removing the token_type_ids because we don't need them
        try:
            inputs.pop('token_type_ids')
        except KeyError:
            pass
        labels = inputs['input_ids'].clone()
        # Generating the labels, because the model can only predict F-Terms but also can interpret Text-Tokens as input, 
        # The maximum token idx is 50000 higher than the maximum output_idx
        labels = labels - 50000
        # All text tokens have token_idx below 50000 after substracting 50000 they are negative and 
        # are now set to -100 to ignore them when the loss is computed
        labels[labels<0] = -100
        # generating the output of the model
        # It is a dict of 'loss', 'logits' and 'past_key_values'
        outputs = model(**inputs, output_attentions=False, output_hidden_states=False, return_dict=True, labels=labels)
        loss = outputs['loss']
        print('loss', loss)
        return (loss, outputs) if return_outputs else loss
        

# Finding the Optimal Batch-Size

# Extracting the number of tokens per sample

file_indices = np.arange(train_dataset.l)
empty_df = pd.DataFrame(file_indices, columns=['Files'])
lengths_df = dd.from_pandas(empty_df, chunksize=100000)

def load_file(index):
    with open(f'{train_dataset.data_folder}/{index}.txt', 'r', encoding='utf-8') as f:
        txt = f.read()
    tokens = train_dataset.tokenizer(txt)
    return len(tokens['input_ids'])

lengths_df['Lengths'] = lengths_df['Files'].map(load_file)
lengths_df.head()

# Finding the longest sample
n_tokens = 0
max_l = 0
for i, l in enumerate(lengths_df['Lengths']):
    n_tokens += l
    if l > max_l:
        max_l = l
    if i%1000 == 0:
        print(f'Processed {i} samples, maximum found lengths = {max_l}, number of total tokens = {n_tokens}', end= '\r')
    

class DummyDataset(Dataset):
    def __init__(self, l):
        self.l = 100
        self.sample = {'input_ids': [int(x) for x in np.random.randint(50000, 400000, size=l)],  'attention_mask': [1 for _ in range(l)]}

    def __len__(self):
        return self.l

    def __getitem__(self, idx):
        return self.sample
        

max_l = 1279

# Iterating over increcingly higher batch-sizes untill the maximum is found, starting batch_size is 100
dummy_ds = DummyDataset(max_l)
for batch_size in range(4, 1, -1):
    print(f'Testing batch_size {batch_size}', end='\r')
    torch.cuda.empty_cache()
    
    training_args = TrainingArguments(
        output_dir=output_dir,          # output directory
        num_train_epochs=1,              # total # of training epochs
        per_device_train_batch_size=batch_size,    # batch size per device during training
        save_strategy=save_strategy,
        logging_strategy=logging_strategy,
        evaluation_strategy=evaluation_strategy,
        learning_rate=learning_rate,
        weight_decay=weight_decay)

    trainer = CustomTrainer(model=model, args=training_args, train_dataset=dummy_ds, eval_dataset=dummy_ds, data_collator=DataCollatorWithPadding(tokenizer, return_tensors='pt'))

    try: 
        trainer.train()
    except OutOfMemoryError:
        continue
        


batch_size = 10

for n_tokens in range(450, 1000, 10):
    print(f'Maximum number of tokens per sample for batchsize {batch_size} = {n_tokens}')

    max_l = n_tokens

    torch.cuda.empty_cache()
    dummy_ds = DummyDataset(max_l)
    training_args = TrainingArguments(
            output_dir=output_dir,          # output directory
            num_train_epochs=1,              # total # of training epochs
            per_device_train_batch_size=batch_size,    # batch size per device during training
            save_strategy=save_strategy,
            logging_strategy=logging_strategy,
            evaluation_strategy=evaluation_strategy,
            learning_rate=learning_rate,
            weight_decay=weight_decay)
    
    trainer = CustomTrainer(model=model, args=training_args, train_dataset=dummy_ds, eval_dataset=dummy_ds, data_collator=DataCollatorWithPadding(tokenizer, return_tensors='pt'))
    
    trainer.train()

max_bs_for_1279 = 3
maximum tokens per sample for batch size 4 = 1034
maximum tokens per sample for batch size 5 = 864
maximum tokens per sample for batch size 6 = 740
maximum tokens per sample for batch size 7 = 650
maximum tokens per sample for batch size 8 = 570 
maximum tokens per sample for batch size 9 = 520
maximum tokens per sample for batch size 10 = 470

# Training the Model

In [ ]:
# The TrainingArguments class is a class which stores multiple parameters for the Custom-trainer of the model.

training_args = TrainingArguments(
    output_dir=output_dir,          
    logging_dir=log_folder,
    num_train_epochs=num_train_epochs,              # total # of training epochs
    per_device_train_batch_size=per_device_train_batch_size,    # batch size per device during training
    per_device_eval_batch_size=per_device_eval_batch_size,
    save_strategy=save_strategy,
    logging_strategy=logging_strategy,
    evaluation_strategy=evaluation_strategy,
    eval_steps=eval_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    seed=seed,
    dataloader_num_workers=dataloader_num_workers, 
    fp16=fp16,
    torch_compile=torch_compile
)


In [ ]:
trainer = CustomTrainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=validation_dataset, data_collator=DataCollatorWithPadding(tokenizer, return_tensors='pt'))

In [ ]:
trainer.train()

In [ ]:
import transformers
print(transformers.__version__)
print(python.__version__)

In [ ]:
sys.path

In [ ]:
torch.cuda.empty_cache()

In [ ]:
log_folder = 'data/models/gal_125_1/logs'
os.makedirs(log_folder, exist_ok=True)

# Folder at which all pickle files are stored. This folder is fixed for this project and should not be changed
dump_dir = r'PK_DUMP'

# Model parameters 
'''
mini	125 M
base	1.3 B
standard	6.7 B
large	30 B
huge	120 B'''
base_model_name = 'mini'

# All new Torch-objects will be by default in this dtype
default_dtype = torch.float16
torch.set_default_dtype(default_dtype)

# Default device on which the model will be loaded
default_device = 'cuda:0'

# Number of GPUs the model will be parallelised to 
num_gpus = 1
# If you change 'default_device' to 'cpu', make sure to set num_gpus to zero.
if default_device == 'cpu':
    num_gpus = 0

tensor_parallel = False
n_f_terms = None # Will be calculated

# Training parameters!
output_dir=model_folder
num_train_epochs=3
per_device_train_batch_size=10
per_device_eval_batch_size=10
save_strategy="epoch"
logging_strategy="steps"
evaluation_strategy="steps"
eval_steps = 1000000 # eval_steps = number of samples until evaluation is performed.
eval_steps = int(eval_steps/per_device_train_batch_size)
learning_rate=5e-5
weight_decay=0.0
seed = 42

# This that could improve performance
dataloader_num_workers = 4
torch_compile = True
# V-Ram reduction
fp16=True